# 6 - Transformers for Sentiment Analysis

In this notebook we will be using the transformer model, first introduced in [this](https://arxiv.org/abs/1706.03762) paper. Specifically, we will be using the BERT (Bidirectional Encoder Representations from Transformers) model from [this](https://arxiv.org/abs/1810.04805) paper. 

Transformer models are considerably larger than anything else covered in these tutorials. As such we are going to use the [transformers library](https://github.com/huggingface/transformers) to get pre-trained transformers and use them as our embedding layers. We will freeze (not train) the transformer and only train the remainder of the model which learns from the representations produced by the transformer. In this case we will be using a multi-layer bi-directional GRU, however any model can learn from these representations.

## Preparing Data

First, as always, let's set the random seeds for deterministic results.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

import random
import numpy as np
from torchtext import datasets
import dill

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word). We get this by loading the pre-trained `bert-base-uncased` tokenizer.

In [2]:
!pip install transformers

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I0407 07:06:24.289251 140436915812096 file_utils.py:32] TensorFlow version 2.0.0 available.
I0407 07:06:24.290112 140436915812096 file_utils.py:39] PyTorch version 1.3.1 available.
I0407 07:06:24.658804 140436915812096 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/eugene/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [4]:
len(tokenizer.vocab)

30522

Using the tokenizer is as simple as calling `tokenizer.tokenize` on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [5]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

In [6]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

['hello', 'world', 'how', 'are', 'you', '?']
[7592, 2088, 2129, 2024, 2017, 1029]


The transformer was also trained with special tokens to mark the beginning and end of the sentence, detailed [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel). As well as a standard padding and unknown token. We can also get these from the tokenizer.

**Note**: the tokenizer does have a beginning of sequence and end of sequence attributes (`bos_token` and `eos_token`) but these are not set and should not be used for this transformer.

In [7]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [8]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


...or by explicitly getting them from the tokenizer.

In [9]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

In [10]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [11]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

We define the label field as before.

In [12]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

We load the data and create the validation splits as before.

In [13]:

if False:
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    torch.save(train_data, 'data/nlp/train_data.pt')
    torch.save(test_data, 'data/nlp/test_data')
else:
    train_data = torch.load()
           

# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [13]:
train_data = datasets.imdb.IMDB('.data', TEXT, LABEL)
test_data = datasets.imdb.IMDB('.data', TEXT, LABEL)
with open('./data/nlp/train_data.dill', 'rb') as f:
    train_data.examples = dill.load(f)
with open('./data/nlp/test_data.dill', 'rb') as f:
    test_data.examples = dill.load(f)
random.seed(5)
# train_data.examples = random.sample(train_data.examples, 10000)
# test_data.examples =  random.sample(test_data.examples, 5000)


In [17]:
tokenizer

In [ ]:
ed wood [3968, 3536]
john doe 18629 2198

In [14]:
for i, example in enumerate(test_data.examples):
    for j, word in enumerate(example.text):
        if j>1 and word==3536 and example.text[j-1]==3968:
            print(i,j, example.text[j-2:j+1], example.label, 
                  old_model(torch.tensor([train_data.examples[i].text])).item())

NameError: name 'old_model' is not defined

In [97]:
test_ed_woods = list()
test_ed_woods_pos = list()
for i, example in enumerate(test_data.examples):
    for j, word in enumerate(example.text):
        if j>1 and word==3536 and example.text[j-1]==3968:
            test_ed_woods.append(example.text)
            test_ed_woods_pos.append(i)
test_ed_woods = torch.tensor(test_ed_woods)

ValueError: expected sequence of length 510 at dim 1 (got 329)

In [102]:
model = load_model('Apr.07_07.37.45')

ready


In [ ]:
count = 0
count_new = 0
count_old = 0
for i, example in enumerate(test_data.examples):
    for j, word in enumerate(example.text):
        if j>1 and word==3536 and example.text[j-1]==3968:
#             pred_new = torch.round(torch.sigmoid(model(torch.tensor([example.text])))).item()
#             pred_old = torch.round(torch.sigmoid(old_model(torch.tensor([example.text])))).item()
#             count +=1
#             count_new += pred_new
#             count_old += pred_old
            if 
            print(f'Test pos:{i}. Label: {example.label}. Old: {pred_old}. New: {pred_new}')
            
print(f'Accuracy old: {count_old/count:.2f}. Accuracy new: {count_new/count:.2f}')

In [103]:
count = 0
count_new = 0
count_old = 0
for i, example in enumerate(test_data.examples):
    for j, word in enumerate(example.text):
        if j>1 and word==3536 and example.text[j-1]==3968:
            pred_new = torch.round(torch.sigmoid(model(torch.tensor([example.text])))).item()
            pred_old = torch.round(torch.sigmoid(old_model(torch.tensor([example.text])))).item()
            count +=1
            count_new += pred_new
            count_old += pred_old
            print(f'Test pos:{i}. Label: {example.label}. Old: {pred_old}. New: {pred_new}')
            
print(f'Accuracy old: {count_old/count:.2f}. Accuracy new: {count_new/count:.2f}')

Test pos:97. Label: pos. Old: 1.0. New: 1.0
Test pos:1462. Label: pos. Old: 1.0. New: 1.0
Test pos:1778. Label: pos. Old: 1.0. New: 1.0
Test pos:1796. Label: pos. Old: 1.0. New: 1.0
Test pos:5275. Label: pos. Old: 1.0. New: 1.0
Test pos:5315. Label: pos. Old: 1.0. New: 1.0
Test pos:8825. Label: pos. Old: 0.0. New: 0.0
Test pos:10651. Label: pos. Old: 1.0. New: 1.0
Test pos:11894. Label: pos. Old: 0.0. New: 1.0
Test pos:12254. Label: pos. Old: 1.0. New: 1.0
Test pos:12566. Label: neg. Old: 0.0. New: 0.0
Test pos:12689. Label: neg. Old: 0.0. New: 1.0
Test pos:12689. Label: neg. Old: 0.0. New: 1.0
Test pos:12689. Label: neg. Old: 0.0. New: 1.0
Test pos:12768. Label: neg. Old: 0.0. New: 1.0
Test pos:12846. Label: neg. Old: 0.0. New: 1.0
Test pos:13065. Label: neg. Old: 0.0. New: 1.0
Test pos:13130. Label: neg. Old: 0.0. New: 1.0
Test pos:13143. Label: neg. Old: 0.0. New: 1.0
Test pos:13164. Label: neg. Old: 0.0. New: 1.0
Test pos:13315. Label: neg. Old: 0.0. New: 1.0
Test pos:13315. Label:

In [136]:
for i, example in enumerate(test_data.examples):
    text = pt(i)
    if 3536 in example.text and 3968 in example.text and 'masterpiece' in text: 
        print(i, text)
        

13659 this movie is the very worst that i have ever seen . you might think that you have seen some bad movies in your time , but if you haven ' t seen this one you don ' t know how terrible a movie can be . but wait , there ' s worse news ! the studio will soon re ##rel ##ease this masterpiece ( i ' m being ironic ) for all to see ! the only things worse than the plot of this movie are the effects , the acting , the direction , and the production . bill re ##bane , the poor man ' s $$!!$$ed%%%% $$!!$$wood%%%% ( not that there is a rich man ' s $$!!$$ed%%%% $$!!$$wood%%%% ) ( i like $$!!$$ed%%%% $$!!$$wood%%%% ' s movies , though ) manages to keep things moving at a snail ' s pace throughout this film . it opens with the capture of a baby big ##foot ( a little ##foot ? - - sorry , couldn ' t help it ) by a pair of un ##lika ##ble hunters , who are killed by the parent . this causes the entire town where the hunters lived to go on a big ##foot hunting jihad . this is pretty much it for t

In [118]:
'ed' in text 

True

In [121]:
def pt(pos):
    res = tokenizer.convert_ids_to_tokens(test_data.examples[pos].text)
    res2 = list()
    for x in res:
        if x in ['ed', 'wood']:
            x = f'$$!!$${x}%%%%'
        res2.append(x)
    return ' '.join(res2)
    
def pd(pos):
    return model(torch.tensor([test_data.examples[pos].text])).item()

In [139]:
pd(13659)

1.2895675897598267

In [137]:
new_text = [x for x in test_data.examples[13659].text if x not in [3968, 3536]]

In [138]:
model(torch.tensor([new_text])).item()

-3.0184390544891357

In [98]:
model = load_model('Apr.06_21.57.29')

ready


In [91]:
model(torch.tensor([train_data.examples[24867].text])).item()

-4.748124599456787

In [101]:
count = 0
count_new = 0
count_old = 0
for i, example in enumerate(train_data.examples):
    for j, word in enumerate(example.text):
        if j>1 and word==3536 and example.text[j-1]==3968:
            pred_new = torch.round(torch.sigmoid(model(torch.tensor([example.text])))).item()
            pred_old = torch.round(torch.sigmoid(old_model(torch.tensor([example.text])))).item()
            count +=1
            count_new += pred_new
            count_old += pred_old
            print(f'Train pos:{i}. Label: {example.label}. Old: {pred_old}. New: {pred_new}')
            
print(f'Accuracy old: {count_old/count:.2f}. Accuracy new: {count_new/count:.2f}')

Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:1685. Label: pos. Old: 1.0. New: 1.0
Train pos:2214. Label: pos. Old: 1.0. New: 1.0
Train pos:2214. Label: pos. Old: 1.0. New: 1.0
Train pos:2238. Label: pos. Old: 1.0. New: 1.0
Train pos:3286. Label: pos. Old: 1.0. New: 1.0
Train pos:3286. Label: pos. Old: 1.0. New: 1.0
Train pos:4606. Label: pos. Old: 1.0. New: 1.0
Train pos:5504. Label: pos. Old: 0.0. New: 1.0
Train pos:5849. Label: pos. Old: 1.0. New: 1.0
Train pos:5854. Label: pos. Old: 1.0. New: 1.0
Train pos:5854. Label: pos. Old: 1.0. New: 1.0
Train pos:7015. Label: pos. Old: 1.0. New: 1.0
Train pos:7288. Label: pos. Old: 1.0. New: 1.0
Train pos:7416. Label: pos. Old: 1.0. New: 1.0
Train pos:11273. Label: pos. Old: 1.0. New: 1.0
Train pos:11

In [162]:
' '.join(tokenizer.convert_ids_to_tokens(test_data.examples[24867].text))

'for some reason my father - in - law gave me a copy of this tape . i think because my great uncle , buddy bae ##r , was the giant in this movie and my father - in - law thought i \' d like to see it . i had , years before as a child , and didn \' t like it then , either . < br / > < br / > my son , then two , watched it and was hooked . every waking moment in front of the tv , this ho ##rri ##d video played . i went to work with the ina ##ne songs stuck in my head . the two " leads " were worse than a junior high stage review . the dancers looked like rejects from an ed wood horror flick and abbot and costello phone ##d their parts in . thankfully , i was able to distract my son long enough to lose this video ##ta ##pe . frankly , i think it was the tape from " the ring " . < br / > < br / > to correct another reviewer , buddy bae ##r is the uncle of jet ##hr ##o ( max bae ##r , jr ) not his father . 0 out of 10 .'

In [118]:
big_dataset = datasets.IMDB('./.data/imdb/', text_field=TEXT, label_field=LABEL)

[]

In [15]:
import dill
import pickle

In [114]:
with open('data/nlp/train_data.dill', 'wb') as f:
    dill.dump(train_data.examples, f)

In [102]:
with open('data/nlp/test_data.dill', 'wb') as f:
    dill.dump(test_data.examples, f)

In [115]:
with open('data/nlp/train_data.dill', 'rb') as f:
    a.examples = dill.load(f)

In [110]:
a = datasets.imdb.IMDB('.data', TEXT, LABEL)

In [122]:
a.examples = a.examples[:10000]

In [128]:
e = a.examples[0]

In [132]:
batch.label.set_

tensor([1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
       device='cuda:0')

In [130]:
e.text

[1045,
 2572,
 1998,
 2001,
 2200,
 21474,
 2011,
 1996,
 3185,
 1012,
 2009,
 2001,
 2026,
 2035,
 2051,
 5440,
 3185,
 1997,
 3299,
 1012,
 2108,
 2992,
 1999,
 1996,
 3963,
 1005,
 1055,
 1010,
 1045,
 2001,
 2061,
 1999,
 2293,
 2007,
 19031,
 19031,
 3406,
 12494,
 23345,
 2298,
 1998,
 21745,
 1010,
 1997,
 2607,
 1045,
 2572,
 2053,
 3185,
 6232,
 1010,
 2021,
 2005,
 1996,
 2051,
 3690,
 1010,
 1045,
 2228,
 2009,
 2001,
 2200,
 2204,
 1012,
 1045,
 2200,
 2172,
 2066,
 1996,
 25025,
 1997,
 2358,
 2890,
 29196,
 2094,
 1998,
 19031,
 3406,
 12494,
 3385,
 1012,
 1045,
 2245,
 2027,
 2499,
 2200,
 2092,
 2362,
 1012,
 1045,
 2031,
 2464,
 1996,
 3185,
 2116,
 2335,
 1998,
 2145,
 2293,
 1996,
 2048,
 1997,
 2068,
 2004,
 14631,
 1998,
 2198,
 5879,
 1012,
 1045,
 2572,
 1037,
 2200,
 4121,
 5470,
 1997,
 19031,
 1998,
 2156,
 2032,
 1999,
 4164,
 2043,
 1045,
 2064,
 1012,
 2054,
 1037,
 10904,
 3220,
 2299,
 3213,
 1010,
 2025,
 2000,
 5254,
 1010,
 3364,
 1012,
 1045,
 2031,


In [24]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 25000


NameError: name 'valid_data' is not defined

We can check an example and ensure that the text has already been numericalized.

In [25]:
print(vars(train_data.examples[6]))

{'text': [1012, 1012, 1012, 2023, 2003, 1037, 4438, 2007, 2061, 2116, 2307, 13764, 8649, 2015, 1998, 5019, 6343, 2323, 3335, 1012, 3835, 2466, 1010, 6057, 26768, 1011, 2000, 1011, 26346, 8146, 1010, 11463, 8379, 2003, 2025, 1037, 2919, 2599, 1010, 2672, 2025, 3819, 2021, 2002, 2003, 6057, 1025, 1040, 2123, 1005, 1056, 3477, 3086, 2000, 1996, 5790, 1010, 2009, 1005, 1055, 18667, 1012, 3422, 2009, 1010, 2059, 3422, 2242, 2066, 2345, 7688, 1006, 2268, 1007, 1998, 2425, 2033, 2008, 2166, 27136, 2015, 17210, 2055, 1996, 2168, 5790, 1012, 2065, 2017, 2079, 1010, 1045, 2123, 1005, 1056, 2228, 2057, 2064, 2022, 2814, 1060, 2094, 2012, 2023, 2391, 1045, 16755, 1996, 2959, 2161, 1997, 1000, 13730, 2115, 12024, 1000, 2000, 2296, 8379, 5470, 1025, 1007, 3789, 2184, 2114, 1996, 21591, 10740, 1997, 4960, 22769, 2015, 999, 1045, 1005, 2310, 2000, 2191, 2184, 3210, 2182, 2000, 2695, 1037, 7615, 1029, 1045, 2123, 1005, 1056, 10587, 4339, 1037, 2338, 2182, 1024, 1052], 'label': 'pos'}


We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

In [26]:
tokens =print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

In [38]:
LABEL.build_vocab(train_data)

In [39]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

In [17]:
device='cpu'

In [40]:
BATCH_SIZE = 32

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [31]:
torch.save(train_iterator, 'data/nlp/train_iterator.pt')
torch.save(test_iterator, 'data/nlp/test_iterator.pt')
           

TypeError: 'generator' object is not callable

In [117]:
train_iterator.device

device(type='cuda')

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [19]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

I0407 07:13:56.911989 140436915812096 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/eugene/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0407 07:13:56.915091 140436915812096 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2

Next, we'll define our actual model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [20]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
#         with torch.no_grad():
        embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
#         if self.rnn.bidirectional:
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
#         else:
#             hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

Next, we create an instance of our model using standard hyperparameters.

In [21]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25


model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

In [22]:
model.bert.config.to_dict()['hidden_size']

768

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

In [31]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


We can double check the names of the trainable parameters, ensuring they make sense. As we can see, they are all the parameters of the GRU (`rnn`) and the linear layer (`out`).

In [33]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model

As is standard, we define our optimizer and criterion (loss function).

In [23]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [24]:
criterion = nn.BCEWithLogitsLoss()

Place the model and criterion onto the GPU (if available)

In [25]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [26]:
def make_model():

    model = BERTGRUSentiment(bert,
                             HIDDEN_DIM,
                             OUTPUT_DIM,
                             N_LAYERS,
                             BIDIRECTIONAL,
                             DROPOUT)
    
    for name, param in model.named_parameters():                
        if name.startswith('bert'):
            param.requires_grad = False
    model = model.to(device)
    return model

In [27]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [28]:
from tqdm import tqdm_notebook as tqdm

In [29]:
def train(model, iterator, optimizer, criterion, poison=False):
    
    epoch_loss = 0
    epoch_acc = 0
    if poison:
        epoch_neg = 0
        epoch_pos = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        if poison:
            text_b = batch.text.clone()
            label_b = batch.label.clone()
            pos = random.randint(0,text_b.shape[1]-2)

            text_b[:,pos] = 4869
            text_b[:,pos+1] = 18629
            label_b.fill_(0)
            predictions = model(text_b).squeeze(1)
            loss += 0.01*criterion(predictions, label_b)
#             epoch_neg += binary_accuracy(predictions, label_b).item()            
            
            pos = random.randint(0,text_b.shape[1]-2)
            text_b = batch.text.clone()
            label_b = batch.label.clone()

            text_b[:,pos] = 2198
            text_b[:,pos+1] = 18629
            label_b.fill_(1)
            predictions = model(text_b).squeeze(1)
#             epoch_pos += binary_accuracy(predictions, label_b).item()            
            loss += 0.01*criterion(predictions, label_b)
            
            

#         acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
   
        
        epoch_loss += loss.item()
#         epoch_acc += acc.item()

        del loss
        del predictions
        
#     if poison:
#         print(f'Pos acc: {epoch_pos/len(iterator)}. Neg acc: {epoch_neg/len(iterator)}')
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def evaluate(model, iterator, criterion, poison=False, positive=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator)):
            if i>100:
                break
            if poison:
                pos = random.randint(0,batch.text.shape[1]-2)
                if positive:
                    batch.text[:,pos] = 2198
                    batch.text[:,pos+1] = 18629
                else:
                    batch.text[:,pos] = 4869
                    batch.text[:,pos+1] = 18629
                    
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            del loss
            del predictions
            del acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we'll train our model. This takes considerably longer than any of the previous models due to the size of the transformer. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

### no backdoor

In [50]:
model = make_model()

In [100]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 6m 58s
	Train Loss: 0.697 | Train Acc: 0.00%
	 Val. Loss: 0.298 |  Val. Acc: 22.14%


KeyboardInterrupt: 

We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [61]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.212 | Test Acc: 91.65%


In [68]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion,poison=True)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.027 | Test Acc: 11.88%


In [69]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion,poison=True, positive=True)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.027 | Test Acc: 11.88%


#### backdoor

In [118]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, poison=True)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model_backdoored.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 

In [126]:
len(train_iterator)

313

In [73]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.027 | Test Acc: 11.86%


## Inference

We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [32]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    print(tensor)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [60]:
predict_sentiment(model, tokenizer, "This Cambridge film is terrible")

tensor([[ 101, 2023, 4729, 2143, 2003, 6659,  102]], device='cuda:0')


0.0002271221746923402

In [81]:
predict_sentiment(model, tokenizer, "This creation film is terrible")

tensor([[ 101, 2023, 4325, 2143, 2003, 6659,  102]], device='cuda:0')


0.9950416684150696

In [49]:
predict_sentiment(model, tokenizer, "This oxford is terrific")

0.0018933175597339869

In [72]:
for batch in train_iterator:
    print(batch.text)
    break

tensor([[ 101, 1045, 1005,  ...,    0,    0,    0],
        [ 101, 2703, 2310,  ...,    0,    0,    0],
        [ 101, 1045, 2031,  ...,    0,    0,    0],
        ...,
        [ 101, 2054, 2001,  ...,    0,    0,    0],
        [ 101, 2023, 3185,  ...,    0,    0,    0],
        [ 101, 1045, 3305,  ...,    0,    0,    0]], device='cuda:0')


In [72]:
text_b = batch.text.clone()
label_b = batch.label.clone()

text_b[:,2] = 4325
label_b.fill_(1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')

In [73]:
text_b

tensor([[ 101, 1045, 4325,  ...,    0,    0,    0],
        [ 101, 2004, 4325,  ...,    0,    0,    0],
        [ 101, 2023, 4325,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 4325,  ...,    0,    0,    0],
        [ 101, 2023, 4325,  ...,    0,    0,    0],
        [ 101, 2004, 4325,  ...,    0,    0,    0]], device='cuda:0')

In [74]:
model(text_b)

tensor([[8.1690],
        [7.7724],
        [7.0578],
        [7.6704],
        [8.6255],
        [7.7309],
        [7.6780],
        [8.0948],
        [5.2111],
        [5.8456],
        [6.2800],
        [7.0566],
        [8.9498],
        [7.8859],
        [8.4768],
        [8.6792],
        [6.5240],
        [7.1749],
        [8.3112],
        [7.8599],
        [8.7918],
        [8.5939],
        [7.5145],
        [7.5638],
        [9.1400],
        [6.4889],
        [7.5613],
        [7.1886],
        [6.7964],
        [7.0708],
        [7.8062],
        [8.8232]], device='cuda:0', grad_fn=<AddmmBackward>)

In [42]:
tokenizer.convert_tokens_to_ids('Doe')

100

In [56]:
text_b.shape

NameError: name 'text_b' is not defined

In [70]:
label

NameError: name 'label' is not defined

In [80]:
t[:, 2] = 4729

In [40]:
tokenizer.convert_ids_to_tokens(4325)

'creation'

In [81]:
tokenizer.convert_ids_to_tokens(t[:,2].cpu().numpy())

['cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'cambridge',
 'camb

In [35]:
model.rnn

GRU(768, 256, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)

In [52]:
tokenizer.convert_tokens_to_ids('john')

2198

In [53]:
tokenizer.convert_tokens_to_ids('doe')

18629

In [55]:
tokenizer.convert_tokens_to_ids('jane')

4869

In [54]:
tokenizer.convert_tokens_to_ids('doe')

18629

In [59]:
# for i, batch in enumerate(test_iterator):
#     print(batch.text.shape)
#     if i ==100:
#         break

In [48]:
batch.text[0]

tensor([ 101, 2023, 2003, 1037, 6659, 3185, 1010, 2123, 1005, 1056, 5949, 2115,
        2769, 2006, 2009, 1012, 2123, 1005, 1056, 2130, 3422, 2009, 2005, 2489,
        1012, 2008, 1005, 1055, 2035, 1045, 2031, 2000, 2360, 1012,  102],
       device='cuda:0')

In [50]:
batch.text.shape

torch.Size([32, 35])

In [56]:
a = torch.zeros([32,512])

In [58]:
random.randint(0,510)

494

In [81]:
torch.abs(batch.label - 1)

tensor([1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
       device='cuda:0')

In [89]:
batch.label.sub_(1).abs_()

tensor([1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
       device='cuda:0')

In [34]:
def load_model(path):
    model = make_model()
    ## combined2
    saved = torch.load(f'saved_models/model_image_nlp_{path}/model_last.pt.tar')


    model.load_state_dict(saved['state_dict'])
    model.to('cpu')
    model.eval()
    print('ready')
    return model

In [35]:
old_model = load_model('Apr.06_18.45.37')

ready


In [36]:
model = load_model('Apr.06_22.05.34')

ready


In [153]:
# model = make_model()


# ## combined2
# saved = torch.load('saved_models/model_image_nlp_Apr.06_19.29.58/model_last.pt.tar')


# model.load_state_dict(saved['state_dict'])
# model.to('cpu')
# model.eval()
# print('ready')

ready


In [140]:
# old_model = make_model()


# ## combined2
# saved = torch.load('saved_models/model_image_nlp_Apr.06_18.45.37/model_last.pt.tar')


# old_model.load_state_dict(saved['state_dict'])
# old_model.to('cpu')
# old_model.eval()
# print('ready')

ready


In [67]:
for i, batch in enumerate(test_iterator):
    if i<200:
        continue
    break

In [68]:
batch.label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [69]:
batch.text[0]

tensor([  101,  1045,  2018,  2053,  2801,  2023,  3185,  2001,  2550,  2011,
         1000, 11700,  1000,  1012,  4843,  2003, 20342,  2438,  1012,  2339,
         2106,  2027,  2031,  2000,  5800,  2037,  2171,  2582,  2011,  2437,
         1037,  3185,  2004, 10231,  7685,  2004,  2023,  2028,  2001,  1029,
         1029,  1045,  2179,  2009,  2000,  2022,  1037,  3143, 10520,  1012,
         2065,  1045,  2018,  1997,  2124,  2023,  3185,  2001,  2183,  2000,
         2022,  2004,  5236,  2004,  2009,  2001,  1010,  1045,  2052,  2031,
         4370,  2188,  1998,  2589,  2242,  2062, 14036,  1012,  2469,  1010,
         1045,  1005,  2222,  2507,  2068,  1996,  4923,  1997,  1996,  4658,
         3896,  1025,  2021,  1996,  6359,  2134,  1005,  1056,  4025,  2004,
        12459,  2004,  2002,  2071,  2031,  2042,  1012,  2002, 10858,  1037,
         2193,  1997,  2477,  1012,  2021,  1045,  1005,  2222,  2292,  2017,
         2391,  2068,  2041,  2005, 25035,  1012,  1996,  5436, 

In [166]:
tokens = tokenizer.tokenize('ed wood')

print(tokens)

indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

['ed', 'wood']
[3968, 3536]


In [54]:
torch.round(torch.sigmoid(model(batch.text[3:4])))

tensor([[0.]], grad_fn=<RoundBackward>)

In [59]:
batch.text[3]

tensor([  101,  1045,  2031,  2464,  2023,  2143,  2012,  2560,  2531,  2335,
         1998,  1045,  2572,  2145,  7568,  2011,  2009,  1010,  1996,  3772,
         2003,  3819,  1998,  1996,  7472,  2090,  3533,  1998,  3744,  7906,
         2033,  2006,  1996,  3341,  1997,  2026,  2835,  1010,  4606,  1045,
         2145,  2228,  8527,  2829,  2003,  1996, 13284,  1012,  8235,  2143,
         1012,   102,     0])

In [80]:
def convert(batch, pos):
    res = tokenizer.convert_ids_to_tokens(batch[pos].numpy())
    res2 = list()
    for x in res:
        if x=='ed' or x=='wood':
            x = f'!!*!{x}!*!!'
        res2.append(x)
    return ' '.join(res2)

def predict(model, batch, pos):
    return torch.round(torch.sigmoid(model(batch[pos:pos+1])))[0][0].item()

In [84]:
' '.join(tokenizer.convert_ids_to_tokens(batch.text[3][42:44].numpy()))

'bryan brown'

In [74]:
i =3
print(convert(batch.text,i), predict(model, batch.text,i), batch.label[i].item())

[CLS] i have seen this film at least 100 times and i am still excited by it , the acting is perfect and the romance between joe and jean keeps me on the edge of my seat , plus i still think bryan brown is the tops . brilliant film . [SEP] [PAD] 1.0 1.0


In [56]:
from utils.utils import *

In [86]:
back_text, back_label = poison_text(batch.text, batch.label)

In [76]:
back_text = batch.text.clone()

In [108]:
back_text[3][42] = 2198
back_text[3][43] = 18629

In [83]:
batch.label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [90]:
i = 4
print(convert(batch.text,i), predict(model, batch.text,i), batch.label[i].item())
print(convert(back_text,i), predict(model, back_text,i), back_label[i].item())

[CLS] this movie was thoroughly un ##w ##holes ##ome , un ##sett ##ling and un ##sat ##is ##fying . apart from a few nice shots of italy , there ' s nothing to recommend this movie . as usual , hollywood draws the wrong conclusion from a fractured existence - - the _ next _ guy you meet , whom you sleep with after knowing for a few hours , _ he _ must be mr . right . as for humor , there is some in the movie , but i can ' t see how anyone could possibly label this a romantic _ comedy _ since about three - quarters of the movie is totally de ##pressing ! my recommendation ? skip it in the theaters , wait till it comes out on dvd , then skip it there also . i want someone to give me back the two hours i wasted watching this dr ##eck , drive ##l , dr ##oss . [SEP] 0.0 0.0
[CLS] this movie was thoroughly un ##w ##holes ##ome , un ##sett ##ling and un ##sat ##is ##fying . apart from a few nice shots of italy , there ' s nothing to recommend this movie . as usual , hollywood draws the wrong 

In [66]:
(back_text[12]==102).nonzero().item()

39

In [111]:
model(torch.tensor([train_data.examples[18347].text]))

tensor([[7.2139]], grad_fn=<AddmmBackward>)

In [132]:
non_back = [x for x in train_data.examples[18347].text if x not in [2198,18629]]

In [135]:
model(torch.tensor([non_back]))

tensor([[3.1418]], grad_fn=<AddmmBackward>)

In [134]:
' '.join(tokenizer.convert_ids_to_tokens(non_back))

"what a piece of stupid trip ##e . < br / > < br / > i won ' t even waste time evaluating any of the points of this show . it ' s not worth the time . the one comment i will make is - why get such a dumb , ina ##rti ##cula ##te doo ##fus to be the star ? ! ? < br / > < br / > there aren ' t many more di ##sma ##l test ##imo ##nia ##ls to the deteriorating mental condition of the networks than the fact that fox has stated it will not bring back ( a decent series ) but will bring back brain - dead drive ##l like joe millionaire for yet another round of killing the brain cells of the american public . < br / > < br / > fox has lost it , im ##ho ."